# SetUp

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm import tqdm_notebook
import concurrent.futures
from multiprocessing import Pool

In [14]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
import random
import json
import itertools

In [15]:
import sys
sys.platform
sys.version

'linux'

'3.6.7 (default, Dec 11 2018, 07:12:38) \n[GCC 4.8.5 20150623 (Red Hat 4.8.5-36)]'

# 初始化代理ip池

## 获取国外ip

In [271]:
import requests
from bs4 import BeautifulSoup
res = requests.get("https://free-proxy-list.net/")
soup = BeautifulSoup(res.text)
ip_address = [i.text for i in soup.select("#proxylisttable td:nth-child(1)")]
ip_port = [i.text for i in soup.select("#proxylisttable td:nth-child(2)")]
ip_pool = list(zip(ip_address,ip_port))
len(ip_pool)
ip_pool[:3]
ip_pool[-3:]


300

[('128.199.222.84', '8080'),
 ('128.199.161.65', '443'),
 ('138.197.155.142', '80')]

[('31.170.51.40', '80'), ('5.202.101.207', '80'), ('185.108.141.74', '8080')]

### 筛选多个可用ip

In [254]:
import telnetlib
ip_pool_=ip_pool[:20] # 总计有300多个ip
def isUseful(ip,port,timeout=2):
    try:
        telnetlib.Telnet(ip, port=port, timeout=timeout)
    except:
        return False
    else:
        return True

def mapFunc(i):
    return list(i)+[isUseful(i[0],i[1],2)]
with Pool(4) as p:
      ip_pool_withStatus = list(tqdm_notebook(p.imap(mapFunc, ip_pool_), total=len(ip_pool_)))

ip_pool_available = [i for i in ip_pool_withStatus if i[-1]] 
print("可用ip有 %s 个" % len(ip_pool_available))
ip_pool_available[:2]
ip_pool_available[-2:]


可用ip有 20 个


[['128.199.222.84', '8080', True], ['128.199.161.65', '443', True]]

[['157.230.57.57', '8080', True], ['45.55.46.222', '8080', True]]

### 筛选一个可用代理
- 如何适配多线程？

In [272]:
ip_iter = iter(ip_pool)
def isUseful(ip,port,timeout=2):
    try:
        telnetlib.Telnet(ip, port=port, timeout=timeout)
    except:
        return False
    else:
        return True
def getAvailableProxy():
    t = next(ip_iter)
    try:
        while(not isUseful(t[0],t[1])):
            t = next(a)
    except StopIteration as e:
        t = []
#     print("可用的为: ",":".join(t))
    return ":".join(t)

ip = getAvailableProxy()
ip

'128.199.222.84:8080'

## 随机ip初始化webdriver

In [111]:
def initDriver(service_args=[],timeout=5):
    driver = webdriver.PhantomJS(service_args=service_args)
    driver.set_page_load_timeout(timeout)
    driver.set_script_timeout(timeout)
    return driver

    
proxy = ":".join(random.choice(ip_pool_available)[:2])
print("使用代理: %s" % proxy)
service_args = ['--proxy=%s'% proxy,'--load-images=false']
driver = initDriver(service_args,6)
try:
    driver.get('http://httpbin.org/ip')
except Exception as e:
    print(e.message)
    
print("检查代理是否生效,当前ip为: %s" % driver.find_element_by_tag_name("body").text)

使用代理: 47.74.8.220:3128
检查代理是否生效,当前ip为: {
  "origin": "47.74.8.220, 47.74.8.220"
}


# 方案一、直接拼接搜索页的URL
- 注意需从首页获取 Key

## 从首页获取Key

In [259]:
import requests
import re
import time
from lxml import etree
resp = requests.get("http://www.google.com")
html = resp.text
key = re.search(r"kEI:\'(.*?)\'", html).group(1)
print(">>> key:\n",key)

>>> key:
 NsGdXKbJE-qP0wKix5ngBA


## 拼接query请求搜索页
- 请求url并保存html文件（debug用）

In [270]:
query = "India"
url = f"https://www.google.com/search?source=hp&ei={key}&q={query}"
print(">>> request on url: \n"+url)
proxies = {"http":getAvailableProxy(),
           "https":getAvailableProxy()}
print(">>> use proxy as follow: \n",proxies)
print(">>> 本机ip:\n",json.loads(requests.get("http://httpbin.org/ip").text.strip()))
print(">>> 代理ip:\n",json.loads(requests.get("http://httpbin.org/ip",proxies=proxies).text.strip()))

header = {"Referer": "https://www.google.com/",
          'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
resp1 = requests.get(url,headers=header,proxies=proxies)
html = etree.HTML(resp1.text)
print(">>> result len:",len(etree.tostring(html).decode()))
with open("googleSearchResult_request.html","w+") as f: f.write(resp1.text)

>>> request on url: 
https://www.google.com/search?source=hp&ei=NsGdXKbJE-qP0wKix5ngBA&q=India
>>> use proxy as follow: 
 {'http': '128.199.222.84:8080', 'https': ''}
>>> 本机ip:
 {'origin': '144.34.217.10, 144.34.217.10'}
>>> 代理ip:
 {'origin': 'unknown, 128.199.222.84, 128.199.222.84'}
>>> result len: 470680


465135

## 查找class为g的div

In [224]:
div_rso = html.xpath("//div[@id='rso']")
div_bkWMgd = div_rso[0].xpath("./div[@class='bkWMgd']")

def find_g(element):
    div_g = element.xpath("./div[@class='g']")
    if(len(div_g)==0):
        div_srg = element.xpath("./div[@class='srg']")
        if(len(div_srg)>0):
            div_g = div_srg[0].xpath("./div[@class='g']")
    return div_g

div_g = []
for el in div_bkWMgd:
    tmp = find_g(el)
    if(len(tmp)>0):
        div_g.extend(tmp)
print(f"目标div (class为g) 有 {len(div_g)} 个")

目标div有 9 个


## 继续解析class为g的div

In [225]:
def find_result(element):
    a_el = element.xpath(".//div[@class='r']")[0].xpath("a")[0]
    title = a_el.xpath(".//h3[@class='LC20lb']")[0].text
    link = a_el.attrib["href"]
    summary = element.xpath(".//div[@class='s']")[0].xpath("string(.)")
    return [title,link,summary]
result = [find_result(i) for i in div_g]

print(">>>> 搜索结果第一个展示：\n"+"\n".join(result[0]))
print(">>>> 搜索结果最后一个展示：\n"+"\n".join(result[-1]))

>>>> 搜索结果第一个展示：
India - Wikipedia
https://en.wikipedia.org/wiki/India
India (ISO: Bhārat), also known as the Republic of India (ISO: Bhārat Gaṇarājya), is a country in South Asia. It is the seventh largest country by area and with ...Calling code‎: ‎+91Demonym(s)‎: ‎IndianLargest city‎: ‎Mumbai‎; 18°58′30″N 72°49′4...Currency‎: ‎Indian rupee‎ (‎₹‎) (‎INR‎)‎History · ‎Geography · ‎Biodiversity · ‎Politics and government
>>>> 搜索结果最后一个展示：
India - Wikitravel
https://wikitravel.org/en/India
India is the largest country in the South Asia Region, located primarily in the center of South Asia, and shares International borders with Pakistan to the ...Country code‎: ‎+91Capital‎: ‎New DelhiLanguage‎: ‎Hindi, English and 21 other official ...Area‎: ‎total: 3,287,263km²; water: 314,070km²; l...


# 方案二、从首页模拟搜索行为
## 初始化Driver

In [231]:
service_args = ['--load-images=false']
dcap = dict(DesiredCapabilities.PHANTOMJS)
dcap["phantomjs.page.settings.userAgent"] = ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36")
driver = webdriver.PhantomJS(service_args=service_args,desired_capabilities=dcap)
driver.implicitly_wait(15)
driver.get("https://www.google.com")

driver.save_screenshot('googleInitial.png')
with open("googleInitial.html","w+") as f: f.write(driver.page_source)

True

215233

## 进入Google首页进行搜索

In [188]:
query = "India"
driver.get("https://www.google.com")
inputElement = driver.find_element_by_id("main").find_element_by_xpath("""//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[1]/input""")
inputElement.click()
inputElement.send_keys(query)
driver.save_screenshot("send_keys.png")
# 触发搜索
driver.find_element_by_xpath("""//*[@id="tsf"]/div[2]/div/div[3]/center/input[1]""").click()

driver.save_screenshot("search.png")
with open("googleSearchedResult_selenium.html","w+") as f:
    f.write(driver.page_source)

True

True

542744

## 搜索结果页DOM树递进查找

### 查找到class为g的div
- id为rso的最外层div
    - class为g的直接子节点
    - 没有为g的直接子节点，则找为srg的直接子节点
        - class为g的直接子节点

In [189]:
divs_rso=driver.find_elements_by_xpath("""//*[@id="rso"]/div""")
print("> rso下的div有 %s 个" % len(divs_rso))
divs_target = [part for part in divs_rso if len(part.find_elements_by_class_name("g"))>0]
print(">> 包含g标签的div有 %s 个" % len(divs_target))

# 第一层子节点查找"g"，没有则进入"srg"查找”g"，查找结果直接返回
def find_g(element):
    result = element.find_elements_by_xpath("./div[@class='g']")
    if(len(result)==0):
        srg = element.find_elements_by_xpath("./div[@class='srg']")
        if(len(srg)>0):
            result = srg[0].find_elements_by_xpath("./div[@class='g']")
    return result

divs_g = []
for i in divs_target:
    findRes=find_g(i)
    if(len(findRes)>0):
        divs_g.extend(findRes)
print(">>> 包含g标签的div中准确的有 %s 个" % len(divs_g))



> rso下的div有 6 个
>> 包含g标签的div有 4 个
>>> 包含g标签的div中准确的有 8 个


### 在class为g的div中继续查找标题、链接、简介

In [190]:
def find_result(element):
    a_el = element.find_element_by_xpath(".//div[@class='r']").find_element_by_tag_name("a")
    title = a_el.find_element_by_xpath(".//h3[@class='LC20lb']").text
    link = a_el.get_attribute("href")
    summary = element.find_element_by_xpath(".//div[@class='s']").text
    return [title,link,summary]
result = [find_result(i) for i in divs_g]
print(">>>> 搜索结果第一个展示：\n"+"\n".join(result[0]))
print(">>>> 搜索结果最后一个展示：\n"+"\n".join(result[-1]))

>>>> 搜索结果第一个展示：
India - Wikipedia
https://en.wikipedia.org/wiki/India
India (ISO: Bhārat), also known as the Republic of India (ISO: Bhārat Gaṇarājya), is a country in South Asia. It is the seventh largest country by area and with ...
Calling code: +91 Demonym(s): Indian
Largest city: Mumbai; 18°58′30″N 72°49′4... Currency: Indian rupee (₹) (INR)
History · Geography · Biodiversity · Politics and government
>>>> 搜索结果最后一个展示：
India - Wikitravel
https://wikitravel.org/en/India
India is the largest country in the South Asia Region, located primarily in the center of South Asia, and shares International borders with Pakistan to the ...
Country code: +91 Capital: New Delhi
Language: Hindi, English and 21 other official ... Area: total: 3,287,263km²; water: 314,070km²; l...


In [191]:
driver.close()

# 流程化

## 【类封装】ip代理类封装

In [1]:
import requests
from bs4 import BeautifulSoup
import telnetlib
import itertools

class Proxy():
    def __init__(self):
        self.abroadProxiesIter = iter([])
        pass
    
    def initDomesticProxies(self):
        pass
    
    def initAbroadProxies(self):
        res = requests.get("https://free-proxy-list.net/")
        soup = BeautifulSoup(res.text)
        ip_address = [i.text for i in soup.select("#proxylisttable td:nth-child(1)")]
        ip_port = [i.text for i in soup.select("#proxylisttable td:nth-child(2)")]
        ip_pool = [a+":"+b for a,b in zip(ip_address,ip_port)]
        print(f"抓取到 {len(ip_pool)} 个 ip:port 用于初始化")
        self.abroadProxiesIter = itertools.cycle(iter(ip_pool))

    @staticmethod
    def isAvailable(proxy,timeout=2):
        ip,port = proxy.split(":")
        try:
            telnetlib.Telnet(ip, port=port, timeout=timeout)
        except:
            return False
        else:
            return True

    def getAvailableProxy(self):
        t = next(self.abroadProxiesIter)
        try:
            while(not Proxy.isAvailable(t)):
                t = next(self.abroadProxiesIter)
        except StopIteration as e:
            t = ""
        return t

    @staticmethod
    def isAvailableOnURL(proxy,url,timeout=2):
        try:
            resp=requests.get(url,proxies={'http':proxy,'https':proxy},timeout=timeout)
            if (resp.status_code==200):
                return True
            else:
                return False
        except Exception as e:
            print(repr(e))
            return False
        
    def getAvailableProxyOnURL(self,url):
        t = self.getAvailableProxy()
        try:
            while(not Proxy.isAvailableOnURL(t,url)):
                t = self.getAvailableProxy()
        except StopIteration as e:
            t = ""
        return t


### TestCase

In [ ]:
proxyObj = Proxy()
proxyObj.initAbroadProxies()

### TestCase2

In [ ]:
proxyObj=Proxy()
proxyObj.initAbroadProxies()


key = GoogleSearch.getKey()
key
url = f"https://www.google.com/search?source=hp&ei={key}&q={query}"
searcher.gl_proxies
proxyObj.isAvailableOnURL(searcher.gl_proxies,url)

t = next(proxyObj.abroadProxiesIter)
try:
    while(not Proxy.isAvailable(t)):
        t
        t = next(proxyObj.abroadProxiesIter)
except StopIteration as e:
    t = ""
proxyObj.getAvailableProxyOnURL(url)
assert False


resp=requests.get(url,headers=searcher.header,proxies=searcher.gl_proxies,timeout=4)
resp.status_code
with open("tmp.html","w+") as f: f.writelines(resp.text)

## 【类封装】Request直接请求

In [2]:
import requests
import re
import time
from lxml import etree
import random
import json

class GoogleSearch():
    def __init__(self):
        self.gl_query = ""
        self.gl_proxies = {}
        self.key = ""
        self.header = {"Referer":"https://www.google.com/",
                       'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
        self.verbosePath = "googleSearchResult_request.html"
      
    def updateProxies(self,proxies):
        self.gl_proxies = proxies

    def checkCurrentIP(self):
        print("\n\n"+">>> use proxy as follow: \n",self.gl_proxies)
        localIP = json.loads(requests.get("http://httpbin.org/ip",timeout=4).text.strip())
        print(">>> 本机ip:\n",localIP)
        try:
            proxyIP = json.loads(requests.get("http://httpbin.org/ip",proxies=self.gl_proxies,timeout=4).text.strip())
            print(">>> 代理ip:\n",proxyIP)
        except Exception as e:
            print(">>> 代理ip:\n",f"代理网络异常，Exception: {repr(e)}")
    
    # 如果已经获取过key，随机概率使用新key（避免行为过于规律被封）
    def getKey(self):
        if(self.key=="" or random.random()>=0.3):
            resp = requests.get("http://www.google.com")
            html = resp.text
            self.key = re.search(r"kEI:\'(.*?)\'", html).group(1)
        return self.key
    
    @staticmethod
    def getKey():
        resp = requests.get("http://www.google.com")
        html = resp.text
        key = re.search(r"kEI:\'(.*?)\'", html).group(1)
        return key
    
    def request(self,query,key,verbose=False):
        url = f"https://www.google.com/search?source=hp&ei={key}&q={query}"
        resp1 = requests.get(url,headers=self.header,proxies=self.gl_proxies,timeout=4)
        if(verbose):
            with open(self.verbosePath,"w+") as f: f.writelines(resp1.text)
        return resp1.text
    
    @staticmethod
    def _find_g(element):
        div_g = element.xpath("./div[@class='g']")
        if(len(div_g)==0):
            div_srg = element.xpath("./div[@class='srg']")
            if(len(div_srg)>0):
                div_g = div_srg[0].xpath("./div[@class='g']")
        return div_g
    
    @staticmethod
    def _find_result(element):
        a_el = element.xpath(".//div[@class='r']")[0].xpath("a")[0]
        title = a_el.xpath(".//h3[@class='LC20lb']")[0].text
        link = a_el.attrib["href"]
        summary = element.xpath(".//div[@class='s']")[0].xpath("string(.)")
        return [title,link,summary]

    @staticmethod
    def _parse(html_inp):
        html = etree.HTML(html_inp)
        # 查找所有class为g的div
        div_rso = html.xpath("//div[@id='rso']")
        div_bkWMgd = div_rso[0].xpath("./div[@class='bkWMgd']")
        div_g = []
        for el in div_bkWMgd:
            tmp = GoogleSearch._find_g(el)
            if(len(tmp)>0):
                div_g.extend(tmp)
        # 解析class为g的div
        result = []
        for i in div_g:
            try:
                result.append(GoogleSearch._find_result(i))
            except Exception as e:
                print(repr(e))
        return result

    def search(self,query,verbose=False):
        key = self.getKey()
        html_res = self.request(query,key,verbose)
        parse_res = GoogleSearch._parse(html_res)
        return parse_res

    def getResult_json(self,query,verbose=False):
        result = self.search(query,verbose)
        result_dictArr = [dict(zip(["title","link","summary","query"],i+[query])) for i in result]
        resultJSON = json.dumps({query:result_dictArr})
        return resultJSON


### TestCase

In [ ]:
searcher = GoogleSearch()
searcher.checkCurrentIP()

searcher.updateProxies({'http':'84.52.108.218:41290','https':'84.52.108.218:41290'})
searcher.checkCurrentIP()

searcher.updateProxies({})
searcher.checkCurrentIP()

query="Article"
result_dictArr = searcher.getResult_json(query,True)
resultJSON = json.dumps({query:result_dictArr})
resultJSON

## 加载数据

In [3]:
with open("/home/zhoutong/tags/title_tag2.txt","r+") as f:
    tagList = [i.strip().split("\t")[0] for i in f.readlines()]
tagList[:3]
len(tagList)

900

In [ ]:
import sqlite3
# 如果文件不存在，会自动在当前目录创建:
conn = sqlite3.connect('test.db')

# >>> 传统数据库操作方式
# 创建一个Cursor:
cursor = conn.cursor()
# 执行一条SQL语句，创建user表:
cursor.execute('create table user (id varchar(20) primary key, name varchar(20))')
# 1. 插入:
cursor.execute('insert into user (id, name) values (\'1\', \'Michael\')')
# 2. 查询:
cursor.execute('select * from user where id=?', ('1',))
values = cursor.fetchall()
# 通过rowcount获得插入的行数:
cursor.rowcount
# 关闭Cursor:
cursor.close()
# 提交事务:
conn.commit()
# 关闭Connection:
conn.close()

## 请求并保存结果

In [7]:
from tqdm import tqdm_notebook
searcher = GoogleSearch()
with open("searchResult.txt","w+") as f:
    for query in tqdm_notebook(tagList):
        print(f"current query at {query}")
        resultJSON = searcher.getResult_json(query,True)
        f.writelines(resultJSON+"\n")

current query at hindi
current query at tv9
current query at abp news
current query at sports tak
IndexError('list index out of range',)
current query at best scene
IndexError('list index out of range',)
current query at preview
current query at full episode
current query at etv plus
IndexError('list index out of range',)
current query at ntv
current query at news tak
current query at odia serial
IndexError('list index out of range',)
current query at hd
current query at tarangtv
IndexError('list index out of range',)
current query at episode
current query at live
current query at etv telugu
current query at promo
current query at abn telugu
current query at tv5 kannada
current query at teluguone
current query at full song
current query at zeebangla
current query at tv5 news
current query at v6 news
current query at full video
current query at exclusive
current query at hmtv
IndexError('list index out of range',)
current query at mazhavil manorama
current query at webisode
current quer

current query at telugu filmnagar
IndexError('list index out of range',)
current query at reasoning by deepak sir
IndexError('list index out of range',)
current query at yash
current query at mg
current query at bhojpuri hit song
IndexError('list index out of range',)
current query at tollywood nagar
IndexError('list index out of range',)
current query at sidharth tv
IndexError('list index out of range',)
current query at latest punjabi songs
current query at bhojpuri songs
IndexError('list index out of range',)
current query at vs records
current query at bk shivani
current query at interview
current query at bjp
current query at cvr news
current query at kunwari bohu
IndexError('list index out of range',)
current query at mammootty
current query at kalinga tv
current query at coming soon
current query at newsqube
current query at dharuvu tv
current query at vikrant gupta
IndexError('list index out of range',)
current query at ishqbaaz
current query at dance+ 4
current query at maths 

current query at big news big debate
current query at live updates
current query at top news from telugu states
current query at latest punjabi songs 2016
current query at bhojpuri songs 2018
current query at colors tv
current query at trending news
current query at unboxing & review
current query at latest bhojpuri songs 2018
current query at bhojpuri hit songs 2019 new
current query at christian devotional songs malayalam 2018
current query at songs
current query at dumbest review
current query at guru tv
current query at all babies channel
current query at hot news
current query at news18 bihar jharkhand live
IndexError('list index out of range',)
current query at haryanvi dance
current query at mim news express
current query at bollywood news
current query at real-life review
IndexError('list index out of range',)
current query at hindi news
current query at top news
current query at serial
current query at live performance
current query at movie review
current query at top 10 news

IndexError('list index out of range',)
current query at kannada latest news
IndexError('list index out of range',)
current query at honest review
current query at sapna dance
current query at aaj ka news
IndexError('list index out of range',)
current query at watch top news stories of the day
current query at business news live
current query at new year party songs
IndexError('list index out of range',)
current query at star plus tv serial news
IndexError('list index out of range',)
current query at 2018 latest jesus songs
current query at chuchu tv surprise eggs toys live stream
current query at unboxing & hands on review
current query at cnbc tv18  live
current query at business news in english
current query at editor's hour_reporter live
IndexError('list index out of range',)
current query at 6 pm telugu news
current query at govt employees news
current query at bhasad news
current query at election news
current query at road test review
current query at news18 punjab/haryana/himach

current query at zee tv ye teri galiyan serial news 2019
IndexError('list index out of range',)
current query at latest hindi entertainment news from bollywood
IndexError('list index out of range',)
current query at zee tv serials news 2019
IndexError('list index out of range',)
current query at new punjabi songs 2012
current query at bhojpuri hit songs 2016 new
current query at news dnn live stream
current query at latest nollywood movies
IndexError('list index out of range',)
current query at today zee tv serial kaliren news 2018
current query at starplus serial news
IndexError('list index out of range',)
current query at suvarna news pre-poll survey in mandya
IndexError('list index out of range',)
current query at finger family songs
current query at christmas songs for kids
IndexError('list index out of range',)
current query at bollywood dance
current query at ultimate movies tribute
current query at dance short film..
current query at ht_t live
current query at gta online
current

In [49]:
import sqlite3